[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rpi-techfundamentals/fall2018-materials/blob/master/10-deep-learning/04-pytorch-mnist.ipynb)


# Installation

In [ ]:
# Install required libraries
!pip install sentence-transformers faiss-cpu transformers

# LLM

## Task: Please obtain your access token from Hugging Face. You can follow the tutorial titled '**Tutorial on Obtaining Access Token from Hugging Face**'. Replace the below hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx with your obtained access token.

In [ ]:
AccessToken = "hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
FlanT5_API_URL = "https://api-inference.huggingface.co/models/google/flan-t5-large"
headers = {"Authorization": "Bearer " + AccessToken}

In [ ]:
import requests

def FlanT5(payload):
    response = requests.post(FlanT5_API_URL, headers=headers, json=payload)
    response = response.json()
    response = response[0].get('generated_text', "Sorry, I couldn't generate a proper response.")
    return response

prompt = { "inputs": f"When was Bournemouth University awarded the Queen’s Anniversary Prize?"}

response = FlanT5(prompt)
print(response)

1858


You can see the above answer is incorrect.

# 2 Retrieval

Please run the code below.

In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

documents = [
    "Bournemouth University is a public university in Bournemouth, England, with its main campus in Poole.",
    "Bournemouth University was founded in 1992, but its predecessor institutions date back to the early 1900s.",
    "Bournemouth University has two campuses: Talbot Campus (main) and Lansdowne Campus.",
    "Bournemouth University has over 18,000 students, including more than 3,000 international students (2022/23).",
    "Bournemouth University is known for its work in the media industries, with graduates contributing to films like Gravity.",
    "In 2023, Bournemouth University received a silver rating in the Teaching Excellence Framework (TEF).",
    "Bournemouth University is divided into four faculties: Science and Technology, Media and Communication, Health and Social Sciences, and the Business School.",
    "Bournemouth university was awarded the Queen’s Anniversary Prize in 2011 for excellence in computer animation teaching.",
    "Bournemouth University's motto is 'Discere Mutari Est,' meaning 'To Learn is to Change'.",
    "Bournemouth University ranks within the top 500 globally in THE World University Rankings 2025.",
    "The Students' Union at Bournemouth University (SUBU) supports over 120 clubs and societies and operates student venues.",
    "Bournemouth University Chancellor: Kate Adie (since 2019); Bournemouth University Vice-Chancellor: Alison Honour (since 2024).",
    "The Department of Computing & Informatics at Bournemouth University has 46 full-time academic staff, 12 visiting fellows, and over 1000 students, including 27 postgraduate researchers.",
    "The Department of Computing & Informatics at Bournemouth University has been offering computing courses for over 20 years and is known for producing highly employable graduates.",
    "Students from the Department of Computing & Informatics at Bournemouth University have excelled in national and European Cyber Security competitions.",
    "Research in the Department of Computing & Informatics at Bournemouth University is structured around five focus areas, emphasizing interdisciplinarity and collaboration with Psychology, Business, Law, Health, and Tourism.",
    "The Department of Computing & Informatics at Bournemouth University has a strong record in national and European-funded research projects.",
    "The Department of Computing & Informatics at Bournemouth University collaborates with industry through consultancy, training, and student placements.",
    "Companies working with the Department of Computing & Informatics at Bournemouth University include CISCO, Google UK, Hewlett Packard, IBM, Intel, Lockheed Martin UK, Microsoft UK, Ministry of Defence, NHS, Oracle, Siemens, and Xerox."
]

# Load Sentence-Transformers model for text embeddings
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Compute document embeddings and build FAISS index
document_embeddings = np.array([embedding_model.encode(doc) for doc in documents])
dimension = document_embeddings.shape[1]  # Get embedding size
faiss_index = faiss.IndexFlatL2(dimension)  # Create FAISS index
faiss_index.add(document_embeddings)  # Add document embeddings

# Compute query embedding
query = "When was Bournemouth University awarded the Queen’s Anniversary Prize?"
query_embedding = embedding_model.encode(query).reshape(1, -1)

# Perform FAISS similarity search
top_k = 2
D, I = faiss_index.search(query_embedding, k=top_k)  # Retrieve top_k relevant documents
retrieved_docs = [documents[idx] for idx in I[0]]

# Construct prompt with retrieved context
context = " ".join(retrieved_docs)
print(context)

Bournemouth university was awarded the Queen’s Anniversary Prize in 2011 for excellence in computer animation teaching. Bournemouth University was founded in 1992, but its predecessor institutions date back to the early 1900s.


## Task: Please encapsulate the above Retrieval code into a function named "def retrieve_relevant_docs".

In [ ]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

def retrieve_relevant_docs(documents, query, model_name="sentence-transformers/all-MiniLM-L6-v2", top_k=2):
    """
    Retrieves the top_k most relevant documents for a given query using FAISS and Sentence-Transformers.

    Parameters:
    documents (list of str): List of documents to search from.
    query (str): The query to search for.
    model_name (str): Name of the Sentence-Transformers model to use for embeddings.
    top_k (int): Number of top relevant documents to retrieve.

    Returns:
    list of str: Retrieved top_k relevant documents.
    """
    # Load Sentence-Transformers model
    embedding_model = SentenceTransformer(model_name)

    # Compute document embeddings
    document_embeddings = np.array([embedding_model.encode(doc) for doc in documents])

    # Create FAISS index
    dimension = document_embeddings.shape[1]
    faiss_index = faiss.IndexFlatL2(dimension)
    faiss_index.add(document_embeddings)

    # Compute query embedding
    query_embedding = embedding_model.encode(query).reshape(1, -1)

    # Perform FAISS similarity search
    D, I = faiss_index.search(query_embedding, k=top_k)
    retrieved_docs = [documents[idx] for idx in I[0]]
    context = " ".join(retrieved_docs)

    return context

# Example usage
documents = [
    "Bournemouth University is a public university in Bournemouth, England, with its main campus in Poole.",
    "Bournemouth University was founded in 1992, but its predecessor institutions date back to the early 1900s.",
    "Bournemouth University has two campuses: Talbot Campus (main) and Lansdowne Campus.",
    "Bournemouth University has over 18,000 students, including more than 3,000 international students (2022/23).",
    "Bournemouth University is known for its work in the media industries, with graduates contributing to films like Gravity.",
    "In 2023, Bournemouth University received a silver rating in the Teaching Excellence Framework (TEF).",
    "Bournemouth University is divided into four faculties: Science and Technology, Media and Communication, Health and Social Sciences, and the Business School.",
    "Bournemouth university was awarded the Queen’s Anniversary Prize in 2011 for excellence in computer animation teaching.",
    "Bournemouth University's motto is 'Discere Mutari Est,' meaning 'To Learn is to Change'.",
    "Bournemouth University ranks within the top 500 globally in THE World University Rankings 2025.",
    "The Students' Union at Bournemouth University (SUBU) supports over 120 clubs and societies and operates student venues.",
    "Bournemouth University Chancellor: Kate Adie (since 2019); Bournemouth University Vice-Chancellor: Alison Honour (since 2024).",
    "The Department of Computing & Informatics at Bournemouth University has 46 full-time academic staff, 12 visiting fellows, and over 1000 students, including 27 postgraduate researchers.",
    "The Department of Computing & Informatics at Bournemouth University has been offering computing courses for over 20 years and is known for producing highly employable graduates.",
    "Students from the Department of Computing & Informatics at Bournemouth University have excelled in national and European Cyber Security competitions.",
    "Research in the Department of Computing & Informatics at Bournemouth University is structured around five focus areas, emphasizing interdisciplinarity and collaboration with Psychology, Business, Law, Health, and Tourism.",
    "The Department of Computing & Informatics at Bournemouth University has a strong record in national and European-funded research projects.",
    "The Department of Computing & Informatics at Bournemouth University collaborates with industry through consultancy, training, and student placements.",
    "Companies working with the Department of Computing & Informatics at Bournemouth University include CISCO, Google UK, Hewlett Packard, IBM, Intel, Lockheed Martin UK, Microsoft UK, Ministry of Defence, NHS, Oracle, Siemens, and Xerox."
]
query = "When was Bournemouth University awarded the Queen’s Anniversary Prize?"
context = retrieve_relevant_docs(documents, query, top_k=2)
print(context)


Bournemouth university was awarded the Queen’s Anniversary Prize in 2011 for excellence in computer animation teaching. Bournemouth University was founded in 1992, but its predecessor institutions date back to the early 1900s.


# 3 Retrieval Augmented Generation

## Task: Please utilise the functions of FlanT5 and retrieve_relevant_docs to implement a Retrieval-Augmented Generation application, allowing the user to ask any question about Bournemouth University.

In [ ]:
import requests
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

def FlanT5(payload):
    response = requests.post(FlanT5_API_URL, headers=headers, json=payload)
    response = response.json()
    response = response[0].get('generated_text', "Sorry, I couldn't generate a proper response.")
    return response

def retrieve_relevant_docs(documents, query, model_name="sentence-transformers/all-MiniLM-L6-v2", top_k=2):

    # Load Sentence-Transformers model
    embedding_model = SentenceTransformer(model_name)

    # Compute document embeddings
    document_embeddings = np.array([embedding_model.encode(doc) for doc in documents])

    # Create FAISS index
    dimension = document_embeddings.shape[1]
    faiss_index = faiss.IndexFlatL2(dimension)
    faiss_index.add(document_embeddings)

    # Compute query embedding
    query_embedding = embedding_model.encode(query).reshape(1, -1)

    # Perform FAISS similarity search
    D, I = faiss_index.search(query_embedding, k=top_k)
    retrieved_docs = [documents[idx] for idx in I[0]]
    context = " ".join(retrieved_docs)

    return context

# Example usage
documents = [
    "Bournemouth University is a public university in Bournemouth, England, with its main campus in Poole.",
    "Bournemouth University was founded in 1992, but its predecessor institutions date back to the early 1900s.",
    "Bournemouth University has two campuses: Talbot Campus (main) and Lansdowne Campus.",
    "Bournemouth University has over 18,000 students, including more than 3,000 international students (2022/23).",
    "Bournemouth University is known for its work in the media industries, with graduates contributing to films like Gravity.",
    "In 2023, Bournemouth University received a silver rating in the Teaching Excellence Framework (TEF).",
    "Bournemouth University is divided into four faculties: Science and Technology, Media and Communication, Health and Social Sciences, and the Business School.",
    "Bournemouth university was awarded the Queen’s Anniversary Prize in 2011 for excellence in computer animation teaching.",
    "Bournemouth University's motto is 'Discere Mutari Est,' meaning 'To Learn is to Change'.",
    "Bournemouth University ranks within the top 500 globally in THE World University Rankings 2025.",
    "The Students' Union at Bournemouth University (SUBU) supports over 120 clubs and societies and operates student venues.",
    "Bournemouth University Chancellor: Kate Adie (since 2019); Bournemouth University Vice-Chancellor: Alison Honour (since 2024).",
    "The Department of Computing & Informatics at Bournemouth University has 46 full-time academic staff, 12 visiting fellows, and over 1000 students, including 27 postgraduate researchers.",
    "The Department of Computing & Informatics at Bournemouth University has been offering computing courses for over 20 years and is known for producing highly employable graduates.",
    "Students from the Department of Computing & Informatics at Bournemouth University have excelled in national and European Cyber Security competitions.",
    "Research in the Department of Computing & Informatics at Bournemouth University is structured around five focus areas, emphasizing interdisciplinarity and collaboration with Psychology, Business, Law, Health, and Tourism.",
    "The Department of Computing & Informatics at Bournemouth University has a strong record in national and European-funded research projects.",
    "The Department of Computing & Informatics at Bournemouth University collaborates with industry through consultancy, training, and student placements.",
    "Companies working with the Department of Computing & Informatics at Bournemouth University include CISCO, Google UK, Hewlett Packard, IBM, Intel, Lockheed Martin UK, Microsoft UK, Ministry of Defence, NHS, Oracle, Siemens, and Xerox."
]

query = "When was Bournemouth University awarded the Queen’s Anniversary Prize?"
context = retrieve_relevant_docs(documents, query, top_k=2)
prompt = {"inputs": f"Based on the following knowledge, answer the question: {context}. Question: {query}"}
rag_response = FlanT5(prompt)
print(rag_response)

2011


Now you can see that the answer is correct by using RAG.

## Task: Please develop a simple user interface in Colab for the above application so that the user can input a question.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def on_submit(change):
    query = question_input.value
    context = retrieve_relevant_docs(documents, query, top_k=2)
    prompt = {"inputs": f"Based on the following knowledge, answer the question: {context}. Question: {query}"}
    response = FlanT5(prompt)
    output_text.value = response

question_input = widgets.Text(placeholder="Enter your question here...")
question_input.observe(on_submit, names='value')
output_text = widgets.Textarea(placeholder="Response will appear here...", disabled=True)

display(question_input, output_text)

Text(value='', placeholder='Enter your question here...')

Textarea(value='', disabled=True, placeholder='Response will appear here...')

## Task: Enhance the above simple application.

Furthermore, you can enhance the above simple application by improving the UI or integrating a more powerful LLM. Additionally, the application is a Retrieval-Augmented Generation chatbot designed for Bournemouth University, but you may also want to explore other areas.

In [ ]:
import requests
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from huggingface_hub import InferenceClient

client = InferenceClient(
  provider="together",
	api_key=AccessToken
)

def DeepSeekR1(context, prompt):
    messages = [
        {"role": "system", "content": "You are a concise, knowledgeable assistant. Answer clearly and directly using the provided context, keeping responses under 64 words."},
        {"role": "system", "content": f"Context: {context}"},
        {"role": "user", "content": prompt}
    ]
    completion = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-R1",
        messages=messages,
        max_tokens=256,
        temperature=0.7,
        top_p=0.9
    )
    return completion.choices[0].message.content

def retrieve_relevant_docs(documents, query, model_name="sentence-transformers/all-MiniLM-L6-v2", top_k=2):

    # Load Sentence-Transformers model
    embedding_model = SentenceTransformer(model_name)

    # Compute document embeddings
    document_embeddings = np.array([embedding_model.encode(doc) for doc in documents])

    # Create FAISS index
    dimension = document_embeddings.shape[1]
    faiss_index = faiss.IndexFlatL2(dimension)
    faiss_index.add(document_embeddings)

    # Compute query embedding
    query_embedding = embedding_model.encode(query).reshape(1, -1)

    # Perform FAISS similarity search
    D, I = faiss_index.search(query_embedding, k=top_k)
    retrieved_docs = [documents[idx] for idx in I[0]]
    context = " ".join(retrieved_docs)

    return context

# Example usage
documents = [
    "Bournemouth University is a public university in Bournemouth, England, with its main campus in Poole.",
    "Bournemouth University was founded in 1992, but its predecessor institutions date back to the early 1900s.",
    "Bournemouth University has two campuses: Talbot Campus (main) and Lansdowne Campus.",
    "Bournemouth University has over 18,000 students, including more than 3,000 international students (2022/23).",
    "Bournemouth University is known for its work in the media industries, with graduates contributing to films like Gravity.",
    "In 2023, Bournemouth University received a silver rating in the Teaching Excellence Framework (TEF).",
    "Bournemouth University is divided into four faculties: Science and Technology, Media and Communication, Health and Social Sciences, and the Business School.",
    "Bournemouth university was awarded the Queen’s Anniversary Prize in 2011 for excellence in computer animation teaching.",
    "Bournemouth University's motto is 'Discere Mutari Est,' meaning 'To Learn is to Change'.",
    "Bournemouth University ranks within the top 500 globally in THE World University Rankings 2025.",
    "The Students' Union at Bournemouth University (SUBU) supports over 120 clubs and societies and operates student venues.",
    "Bournemouth University Chancellor: Kate Adie (since 2019); Bournemouth University Vice-Chancellor: Alison Honour (since 2024).",
    "The Department of Computing & Informatics at Bournemouth University has 46 full-time academic staff, 12 visiting fellows, and over 1000 students, including 27 postgraduate researchers.",
    "The Department of Computing & Informatics at Bournemouth University has been offering computing courses for over 20 years and is known for producing highly employable graduates.",
    "Students from the Department of Computing & Informatics at Bournemouth University have excelled in national and European Cyber Security competitions.",
    "Research in the Department of Computing & Informatics at Bournemouth University is structured around five focus areas, emphasizing interdisciplinarity and collaboration with Psychology, Business, Law, Health, and Tourism.",
    "The Department of Computing & Informatics at Bournemouth University has a strong record in national and European-funded research projects.",
    "The Department of Computing & Informatics at Bournemouth University collaborates with industry through consultancy, training, and student placements.",
    "Companies working with the Department of Computing & Informatics at Bournemouth University include CISCO, Google UK, Hewlett Packard, IBM, Intel, Lockheed Martin UK, Microsoft UK, Ministry of Defence, NHS, Oracle, Siemens, and Xerox."
]

query = "When was Bournemouth University awarded the Queen’s Anniversary Prize?"
context = retrieve_relevant_docs(documents, query, top_k=1)
prompt = query
rag_response = DeepSeekR1(context, prompt)
print(rag_response)

<think>
Okay, the user is asking when Bournemouth University received the Queen’s Anniversary Prize. Let me check the context they provided. It says the award was given in 2011 for excellence in computer animation teaching.

First, I need to confirm that the answer is indeed 2011. Since the context is straightforward, there's no ambiguity here. The user might be looking for a quick fact check or maybe they're compiling information about the university's achievements. 

I should make sure to state the year clearly and mention the reason for the award, which is computer animation teaching. This adds value by explaining why they received it. The user might also be interested in knowing the significance of the award, but since the context doesn't provide that, I'll stick to the given information. 

No need for extra details unless the user asks for more. Just answer concisely. Double-check that the context mentions 2011 and the specific field. Yep, that's all there. Alright, ready to respo

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def on_submit(change):
    query = question_input.value
    context = retrieve_relevant_docs(documents, query, top_k=2)
    prompt = query
    response = DeepSeekR1(context, prompt)
    output_text.value = response

question_input = widgets.Text(placeholder="Enter your question here...")
question_input.observe(on_submit, names='value')
output_text = widgets.Textarea(placeholder="Response will appear here...", disabled=True)

display(question_input, output_text)

Text(value='', placeholder='Enter your question here...')

Textarea(value='', disabled=True, placeholder='Response will appear here...')